### Introdução

Esse notebook visa testar uma Deep Neural Network construida "from scratch" para classificar um dado de uso de cartões de crédito 

#### Importações

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import PIL

In [2]:
#Importando modelo
from DNN.models.custom_model import CustomModel
from DNN.layers.layer import Dense
from DNN.layers.activation import ReLU, Sigmoid
from DNN.training.loss import CrossEntropyLoss
from DNN.training.optimizer import GradientDescent
from DNN.training.trainer import Trainer
from DNN.evaluation.evaluate import evaluate
from DNN.evaluation.cross_validation import CrossValidation

#### Lendo datasets

In [3]:
df = pd.read_csv("../datasets/creditcard.csv")

In [4]:
df.groupby("Class")["Class"].count()

Class
0    284315
1       492
Name: Class, dtype: int64

Fazendo a contagem de linhas para cada classe, vemos que há um número bastante desigual de linhas entre as labels 0 e 1, o que pode tornar o treinamento do modelo enviesado. Para atenuar esse problema, utilizarei um método de amostragem chamado Stratified K-Fold, que consiste na separação de amostras que buscam igualar a proporção entre os dois tipos de labels para cada 'fold'.

In [5]:
#Importando Stratified K-Fold do sklearn
from sklearn.model_selection import StratifiedKFold

In [6]:
#Separando entradas e saídas
X = df.loc[:,df.columns != "Class"].copy()
y = df['Class'].copy()

In [7]:
#Realizando a primeira separação entre set de treinamento e de teste com proporção 0.1 de teste.
skf = StratifiedKFold(n_splits=2)

#Obtendo indexadadores das amostras de treino e de teste
train_idx, test_idx = list(skf.split(X,y))[0][0], list(skf.split(X,y))[0][1]

#Aplicando indexadores às entradas e saidas
X_train, y_train = np.array(X.loc[X.index[train_idx]].copy()), np.array(y[y.index[train_idx]].copy())
X_test, y_test = np.array(X.loc[X.index[test_idx]].copy()), np.array(y[y.index[test_idx]].copy())

In [10]:
#Inicializando modelo, loss_func e otimizador 
model = CustomModel([
    Dense(len(X.columns), 300),
    ReLU(),
    Dense(300,1),
    Sigmoid()])

loss_func = CrossEntropyLoss()
optimizer = GradientDescent(learning_rate=0.01)

In [11]:
trainer = Trainer(model, loss_func, optimizer)
trainer.train(X_train.T, y_train, epochs=200, show = True)

Epoch 1/200, Loss: 20.687467
Epoch 51/200, Loss: 20.687467
Epoch 101/200, Loss: 20.687467
Epoch 151/200, Loss: 20.687467


In [12]:
evaluate(model, X_test.T, y_test,show=True)

ValueError: shapes (300,30) and (142404,30) not aligned: 30 (dim 1) != 142404 (dim 0)